In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jax.random as jr
import jax.tree_util as jtu

from tom.models import OvercookedModel, OvercookedAgent
from jaxmarl.viz.overcooked_visualizer import OvercookedVisualizer

from jaxmarl.environments.overcooked import overcooked_layouts, layout_grid_to_dict
from tom.envs.env_ocv1 import OvercookedV1Env

from tom.planning.rollout import rollout as rollout_nontom
from tom.planning.si import si_policy_search as si_policy_search_nontom

# 1 non-ToM agent in overcooked_v1

A non-ToM pymdp Agent completing the jaxMARL overcooked_v1 task.

Initialise the environment

- W = wall
- O = onion pile
- P = pot
- A = agent
- B = plate pile
- X = delivery station
- empty spaces are empty cells

In [ ]:
custom_layout_grid = """
WWPWW
OA  W
W  AB
WWXWW
"""
layout = layout_grid_to_dict(custom_layout_grid)

# # or if you want to use a pre-set layout
# layout = overcooked_layouts["cramped_room"] # options: cramped_room, asymm_advantages, coord_ring, forced_coord, counter_circuit

num_agents = 2
timesteps = 40

In [ ]:
env = OvercookedV1Env(num_agents, layout, timesteps, initiate_inventory=None)
# env = OvercookedV1Env(num_agents, layout, timesteps, initiate_inventory=["onion", "empty"])

Initialise our pymdp agent

In [ ]:
model = OvercookedModel(env_layout=layout)

In [ ]:
num_agents = 2
agents = OvercookedAgent(model, batch_size = num_agents)

Do the rollout using sophisticated active inference (non-ToM)

In [ ]:
horizon=3
max_nodes = 20000
max_branching = len(model.B[0].batch["actions"])
policy_prune_threshold = 0.0
observation_prune_threshold = 1/64
entropy_stop_threshold = 0.5
efe_stop_threshold = 1000
kl_threshold=-1
prune_penalty = 512
gamma = 1.0
topk_obsspace = 1

# set up the policy search function
tree_search_nontom = si_policy_search_nontom(
        horizon=horizon,
        max_nodes=max_nodes,
        max_branching=max_branching,
        policy_prune_threshold=policy_prune_threshold,
        observation_prune_threshold=observation_prune_threshold,
        entropy_stop_threshold=entropy_stop_threshold,
        efe_stop_threshold=efe_stop_threshold,
        kl_threshold=kl_threshold,
        prune_penalty=prune_penalty,
        gamma=gamma,
        topk_obsspace=topk_obsspace
    )

In [ ]:
key = jr.PRNGKey(1)

last, info_nontom, env_final = rollout_nontom(agents, env, timesteps, key, policy_search=tree_search_nontom)


Adding visualisations and printing information

In [ ]:
def convert_State_sequence(info_State):
    num_timesteps = info_State.time.shape[0]
    
    state_seq = []
    for t in range(num_timesteps):
        state_t = jtu.tree_map(
            lambda x: x[t] if x.ndim == 1 else x[:, t, ...],
            info_State
        )
        state_seq.append(state_t)
    
    return state_seq

In [ ]:
state_seq = convert_State_sequence(info_nontom["env_state"])
viz = OvercookedVisualizer()
viz.animate(state_seq, agent_view_size=5, filename='overcooked_v1_nontom_test.gif', fps=3.0)

In [ ]:
import jax.numpy as jnp

def print_beliefs(model, qs, belief_idx):
    state_factor_names = list(model.B.keys())
    state_factor_name = state_factor_names[belief_idx]
    
    labels = model.B[state_factor_name].batch[state_factor_name]
    
    for agent_idx, agent_beliefs in enumerate(qs[belief_idx]):
        print(f"AGENT {agent_idx} {state_factor_name} BELIEFS:")
        
        for t in range(agent_beliefs.shape[0]):
            most_likely_idx = jnp.argmax(agent_beliefs[t])
            probability = agent_beliefs[t, most_likely_idx]
            print(f"  time {t}: {labels[most_likely_idx]} (prob={probability:.3f})")
        print()

def print_actions(actions):
    action_names = model.B[list(model.B.keys())[0]].batch['actions']
    
    for agent_idx, agent_actions in enumerate(actions):
        print(f"AGENT {agent_idx} ACTIONS:")
        
        for t in range(agent_actions.shape[0]):
            action_idx = agent_actions[t][0] if agent_actions[t].ndim > 0 else agent_actions[t]
            print(f"  time {t}: {action_names[action_idx]}")
        print()


In [ ]:
print_beliefs(model,info_nontom["qs"], belief_idx=0)


In [ ]:
print_actions(info_nontom["action"])

In [ ]:
# from tom.planning.visualize import visualize_plan_tree
# root_idx = None
# visualize_plan_tree(info_nontom, time_idx=0, agent_idx=1, model=model, min_prob=0.0, depth=4, fig_size = (8,10), root_idx=root_idx)